In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
# train = pd.read_csv('DATA/train.csv', header=0, encoding = "ISO-8859-1")
from FEATURES import train

In [9]:
# Prepare to divide the dataset into test, train (lifted from sklearn Iris example)
train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75

# Split data into training and test
train_data, test_data = train[train['is_train']==True], train[train['is_train']==False]

In [10]:
features = train.columns[[7, 8, 9]] # 'exact', 'overlapping_words', 'percentage_overlapping_words'

# clf = RandomForestClassifier(n_jobs=2)
clf = RandomForestClassifier(n_jobs=2, class_weight = 'balanced', oob_score = False, criterion = 'entropy')

y, _ = pd.factorize(train_data['relevance_rounded']) # 0, 1, 2

clf.fit(train_data[features], y)

# Predicting on those features will output predictions that match y
preds = clf.predict(test_data[features])

# target_names = test['relevance_rounded']
target_names = ['1', '2', '3']
out = [target_names[pred] for pred in preds]

# preds = target_names[clf.predict(test[features])]

preds = clf.predict(test_data[features])

# preds.index = range(len(preds))
# pd.crosstab(test['relevance_rounded'], preds, rownames=['actual'], colnames=['preds'])
print(pd.crosstab(test_data['relevance_rounded'], np.asarray(out), rownames=['actual'], colnames=['preds']))

preds      1     2     3
actual                  
1        163   364   723
2       2551  3386  2834
3       4878  2162  1630


In [11]:
ct = pd.crosstab(test_data['relevance_rounded'], np.asarray(out), rownames=['actual'], colnames=['preds'])
print(ct)

preds      1     2     3
actual                  
1        163   364   723
2       2551  3386  2834
3       4878  2162  1630


In [10]:
row = ct.ix[1]
col = ct['1']
print(row)
print(col)

preds
1    152
2    289
3    875
Name: 1.0, dtype: int64
actual
1     152
2    2150
3    4028
Name: 1, dtype: int64


In [21]:
index = list(pd.crosstab(test_data['relevance_rounded'], preds))
total = 0
for i in list(ct):
    total += sum(list(ct[i]))

true_positives, false_positives, true_negatives, false_negatives = 0, 0, 0, 0

for i in zip(list(ct), range(1, len(ct)+1)): 
    print(i)
    row = list(ct.ix[i[1]])
    column = list(ct[i[0]])
    
    tp = row[i[1]-1]
    true_positives  += tp
    false_negatives += sum(row)-tp
    false_positives += sum(column) - tp
    true_negatives = total - tp
    
print(true_positives, false_positives, false_negatives, true_negatives)

('1', 1)
('2', 2)
('3', 3)
5132 13260 13260 15660


In [22]:
f1 = float(2*true_positives) / (2 * true_positives + false_positives + false_negatives)
f1

0.27903436276642019